### IMPORTS

In [1]:
import BBVI
import torch
import matplotlib.pyplot as plt

### EXAMPLE

In [ ]:
data = torch.load('foong_data.pt')
optimizer_params = [0.1, 3, 0.5]               # [learning_rate, patience, factor]
sigma_noise = 0.1
gpu = '1'
n_seed = 0
name = '_SampleELBO'

In [ ]:
BBVI_trained = BBVI.BBVI(data=data, n_neurons=50, n_epoch=10, n_iter=10, n_samples_ELBO=1, sigma_noise=sigma_noise, 
              opti_params=optimizer_params, n_seed=n_seed, gpu=gpu, name=name)

In [ ]:
BBVI.plot_BBVI_Uncertainty(BBVI_trained, data, name, gpu)

### TRAINING LOOP FOR DIFFERENT NUMBERS OF SAMPLING

In [ ]:
data = torch.load('foong_data.pt')
optimizer_params = [0.1, 3, 0.5]               # [learning_rate, patience, factor]
sigma_noise = 0.1
gpu = '1'
n_seed = 0

In [ ]:
for i in [1,5,10,20]:
    name = '_SamplingELBO_'+str(i)
    BBVI_trained = BBVI.BBVI(data=data, n_neurons=50, n_epoch=5, n_iter=10, n_samples_ELBO=i, sigma_noise=sigma_noise, 
              opti_params=optimizer_params, n_seed=n_seed, gpu=gpu, name=name);
    BBVI.plot_BBVI_Uncertainty(BBVI_trained, data, name, gpu);